# Preprocesamiento de los datos - Análisis de suelos
## Importaciones

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de los datos

In [126]:
df = pd.read_csv('data/suelos_original.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_22272\987695195.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/suelos_original.csv')


## Pre-procesamiento de los datos

### Renombrar columnas

In [127]:
oDict = {
    'numfila': 'id',
    'Departamento': 'departamento',
    'Municipio': 'municipio',
    'Cultivo': 'cultivo',
    'Estado': 'estado',
    'Tiempo Establecimiento': 'tiempo_establecimiento',
    'Topografia': 'topografia',
    'Drenaje': 'drenaje',
    'Riego': 'riego',
    'Fertilizantes aplicados': 'fertilizantes',
    'FechaAnalisis': 'fecha',
    'pH agua:suelo 2,5:1,0': 'ph',
    'Materia orgánica (MO) %': 'materia_organica',
    'Fósforo (P) Bray II mg/kg': 'fosforo',
    'Azufre (S) Fosfato monocalcico mg/kg': 'azufre',
    'Acidez (Al+H) KCL cmol(+)/kg': 'acidez',
    'Aluminio (Al) intercambiable cmol(+)/kg': 'aluminio',
    'Calcio (Ca) intercambiable cmol(+)/kg': 'calcio',
    'Magnesio (Mg) intercambiable cmol(+)/kg': 'magnesio',
    'Potasio (K) intercambiable cmol(+)/kg': 'potasio',
    'Sodio (Na) intercambiable cmol(+)/kg': 'sodio',
    'capacidad de intercambio cationico (CICE) suma de bases cmol(+)/kg': 'cice',
    'Conductividad el‚ctrica (CE) relacion 2,5:1,0 dS/m': 'ce',
    'Hierro (Fe) disponible olsen mg/kg': 'hierro_olsen',
    'Cobre (Cu) disponible mg/kg': 'cobre',
    'Manganeso (Mn) disponible Olsen mg/kg': 'manganeso',
    'Zinc (Zn) disponible Olsen mg/kg': 'zinc_olsen',
    'Boro (B) disponible mg/kg': 'boro',
    'Hierro (Fe) disponible doble \xa0cido mg/kg': 'hierro_doble_acido',
    'Cobre (Cu) disponible doble acido mg/kg': 'cobre_doble_acido',
    'Manganeso (Mn) disponible doble acido mg/kg': 'manganeso_doble_acido',
    'Zinc (Zn) disponible doble \xa0cido mg/kg': 'zinc_doble_acido', 
    'Secuencial': 'secuencial'
}
df.rename(columns=oDict, inplace=True)

### Eliminar columnas sin valores

In [128]:
df.drop(columns=['secuencial', 'fecha'], inplace=True)

### Formato de valores en columnas

In [129]:
# TODO capitalizar departamento y municipio
# Cambiar los valores a minúsculas
columnas = [
    'departamento', 'municipio', 'cultivo', 'estado', 'tiempo_establecimiento', 
    'topografia', 'drenaje', 'riego', 'fertilizantes'
]
for columna in columnas:
    df[columna] = df[columna].str.lower()

In [130]:
# Valores faltantes en columnas float
columnas = [
    'azufre', 'acidez', 'aluminio', 'potasio', 'sodio', 'ce','hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido', 
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]
for columna in columnas:
    df[columna] = df[columna].str.replace('ND', 'NaN')

In [131]:
def clean_float_values(value):
    '''Limpia valores en columnas float
    
    Procesa valores string realizando las siguientes acciones:
      - Reemplaza ',' por '.'
      - Reemplaza '..' por '.'
      - Elimina el signo '>' dejando el límite superior como valor actual
      - Reemplaza los límites inferiores por la mitad de su valor
      - Reemplaza valores inválidos por NaN
    '''
    if type(value) != str:
        return value
    
    value = value.replace(',', '.')
    value = value.replace('..', '.')
    value = value.replace('>', '')
    
    if value[0] == '<':
        value = float((value[1:]).strip()) / 2        
    else:
        try:
            float(value)
        except Exception as e:
            print(str(e));
            value = 'NaN'
    return value

# Limpieza de carácteres no numéricos y conversión de valores numéricos a float
columnas = [
    'ph', 'fosforo', 'azufre', 'acidez', 'aluminio', 
    'calcio', 'magnesio', 'potasio', 'sodio', 'ce', 'hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido',
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]

for columna in columnas:
    try:
        df[columna] = df[columna].apply(clean_float_values)
        df[columna] = df[columna].astype(float)
    except Exception as e:
        print(columna, str(e))

could not convert string to float: 'MI'
could not convert string to float: 'MI'


In [132]:
df["topografia"] = df["topografia"].str.replace("error: #n/a", "no indica")

oDict = {
    'error: #n/a': "no indica",
    'buen drenaje': 'bueno',
    'regular drenaje': 'regular',
    'mal drenaje': 'malo'
}
df["drenaje"].replace(oDict, inplace=True)

In [133]:
indices = df[
    (df["cultivo"] == "no indica") & 
    (df["estado"] == "no indica") & 
    (df["tiempo_establecimiento"] == "no indica")].index
df.drop(index=indices, axis=0, inplace=True)

In [134]:
def clean_string_values(value):
    '''
    '''
    # if type(value) != str:
    #     return value
    
    value = value.replace(' ', '_')
    return value

# Limpieza de espacios en blanco
columnas = [
    'estado', 'tiempo_establecimiento', 'topografia', 'drenaje', 'riego']

for columna in columnas:
    try:
        df[columna] = df[columna].apply(clean_string_values)
    except Exception as e:
        print(columna, str(e))

In [135]:
df.columns

Index(['id', 'departamento', 'municipio', 'cultivo', 'estado',
       'tiempo_establecimiento', 'topografia', 'drenaje', 'riego',
       'fertilizantes', 'ph', 'materia_organica', 'fosforo', 'azufre',
       'acidez', 'aluminio', 'calcio', 'magnesio', 'potasio', 'sodio', 'cice',
       'ce', 'hierro_olsen', 'cobre', 'manganeso', 'zinc_olsen', 'boro',
       'hierro_doble_acido', 'cobre_doble_acido', 'manganeso_doble_acido',
       'zinc_doble_acido'],
      dtype='object')

In [136]:
# Crear columnas dummy a partir de la columna topografia
columnas = [
    'estado', 'tiempo_establecimiento', 'topografia', 'drenaje', 'riego']

for columna in columnas:
    df = pd.get_dummies(df, columns=[columna])

# Ajustar la dummificación de las columnas compuestas
columnas = {
    'topografia_ondulado_y_pendiente' : ['topografia_ondulado', 'topografia_pendiente'],
    'topografia_plano_y_ondulado' : ['topografia_plano', 'topografia_ondulado'],
    'topografia_plano_y_pendiente' : ['topografia_plano', 'topografia_pendiente'],
    'riego_aspersión_y_goteo': ['riego_aspersión', 'riego_goteo'],
    'riego_aspersión_y_gravedad': ['riego_aspersión', 'riego_gravedad'],
    'riego_goteo_y_gravedad': ['riego_goteo', 'riego_gravedad'],
}
for key, value in columnas.items():
    indices = df[(df[key] == 1)].index
    df.loc[indices, value[0]] = 1
    df.loc[indices, value[1]] = 1
    df.drop(columns=[key], inplace=True)

## Importar el conjunto de datos de las estaciones metereológicas

In [137]:
df2 = pd.read_csv('data/estaciones_preprocesado.csv')
df2.head()

,departamento,municipio,ubicacion,altitud
0,nariño,ipiales,"(0.81378611, -77.66197778)",2582
1,nariño,contadero,"(0.93030556, -77.49119444)",2450
2,nariño,túquerres,"(1.07061111, -77.63688889)",3120
3,putumayo,mocoa,"(1.08288889, -76.66711111)",760
4,amazonas,puerto nariño,"(-3.78030556, -70.36263889)",158


In [138]:
## Combinar los dos dataframes

In [139]:
df_merged=df.merge(df2, how="inner", suffixes=(False, False))
df_merged

,id,departamento,municipio,cultivo,fertilizantes,ph,materia_organica,fosforo,azufre,acidez,...,riego_cañón,riego_goteo,riego_gravedad,riego_manguera,riego_microaspersión,riego_no_cuenta_con_riego,riego_no_indica,riego_por_inundación,ubicacion,altitud
0,NaN,cundinamarca,funza,uchuva,no indica,5.66,9.706679,5.622299,4.345550,NaN,...,0,0,0,0,0,0,1,0,"(4.73333333, -74.16666667)",2560
1,NaN,cundinamarca,funza,uchuva,no indica,5.66,9.706679,5.622299,4.345550,NaN,...,0,0,0,0,0,0,1,0,"(4.73625, -74.15733333)",2560
2,NaN,cundinamarca,funza,uchuva,no indica,5.66,9.706679,5.622299,4.345550,NaN,...,0,0,0,0,0,0,1,0,"(4.75905556, -74.20516667)",2560
3,NaN,cundinamarca,funza,calas,no indica,5.30,11.021021,84.804303,59.287709,0.501835,...,0,0,0,0,0,0,0,0,"(4.73333333, -74.16666667)",2560
4,NaN,cundinamarca,funza,calas,no indica,5.30,11.021021,84.804303,59.287709,0.501835,...,0,0,0,0,0,0,0,0,"(4.73625, -74.15733333)",2560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200180,NaN,cauca,timbiquí,plátano,no indica,4.98,1.890000,4.350000,10.500000,2.420000,...,0,0,0,0,0,0,1,0,"(2.87180556, -77.62166667)",30
200181,NaN,cauca,timbiquí,plátano,no indica,4.95,4.140000,1.935000,7.040000,1.350000,...,0,0,0,0,0,0,1,0,"(2.77361111, -77.66361111)",30
200182,NaN,cauca,timbiquí,plátano,no indica,4.95,4.140000,1.935000,7.040000,1.350000,...,0,0,0,0,0,0,1,0,"(2.78875, -77.46616667)",40
200183,NaN,cauca,timbiquí,plátano,no indica,4.95,4.140000,1.935000,7.040000,1.350000,...,0,0,0,0,0,0,1,0,"(2.87180556, -77.62166667)",30


## Exportar el conjunto de datos preprocesado

In [140]:
df.to_csv('data/suelos_preprocesado.csv', index=False)

---
Fin del notebook